In [ ]:
!pip install psycopg2-binary sentence-transformers pgvector
!pip install langchain langchain-community


In [1]:
import psycopg2

conn = psycopg2.connect(
    host="aws-1-us-east-1.pooler.supabase.com",
    dbname="postgres",
    user="postgres.znkewdqukpfmzjmkfxea",
    password="planificador123",
    port=5432
)

cur = conn.cursor()
cur.execute("SELECT now();")
print(cur.fetchone())


(datetime.datetime(2026, 2, 10, 4, 21, 51, 204505, tzinfo=datetime.timezone.utc),)


# Creacion de tablas

In [22]:
cur.execute("""
CREATE TABLE IF NOT EXISTS business_functions (
    function_id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT NOT NULL,
    examples TEXT,
    embedding VECTOR(384)
);
""")
conn.commit()


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [5]:
cur.execute("""
CREATE TABLE IF NOT EXISTS query_log (
    query_id SERIAL PRIMARY KEY,
    query_text TEXT,
    query_embedding VECTOR(384),
    selected_function_id INT,
    score FLOAT,
    created_at TIMESTAMP DEFAULT NOW()
);
""")
conn.commit()


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")


/home/mateo/Documentos/Universidad/Camila/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipykernel_36346/124733854.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/mateo/Documentos/Universidad/Camila/back_function_selection/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Funciones

In [ ]:
functions = [

    # =========================
    # FUNCIONES GENERALES
    # =========================
    {
        "name": "saludo",
        "category": "general",
        "description": "Saluda al usuario de forma amable",
        "examples": (
            "hola, buenos días, buenas tardes, buenas noches, "
            "hola qué tal, hey, holaa, buenas, que mas?, que mas ve, como vas"
        )
    },
    {
        "name": "despedida",
        "category": "general",
        "description": "Se despide del usuario de forma cordial",
        "examples": (
            "chao, hasta luego, nos vemos, adiós, "
            "bye, hablamos luego"
        )
    },
    {
        "name": "agradecimiento",
        "category": "general",
        "description": "Responde a mensajes de agradecimiento del usuario",
        "examples": (
            "gracias, muchas gracias, te agradezco, "
            "muy amable, excelente gracias"
        )
    },
    {
        "name": "small_talk",
        "category": "general",
        "description": "Responde mensajes casuales sin intención relacionada con vuelos",
        "examples": (
            "cómo estás, todo bien, qué haces, "
            "cómo te va, estás ahí"
        )
    },

    # =========================
    # FUNCIONES DE VUELOS
    # =========================
    {
        "name": "buscar_vuelos",
        "category": "flight",
        "description": "Busca vuelos disponibles según origen, destino y fechas",
        "examples": (
            "vuelos de quito a guayaquil, pasajes baratos, "
            "quiero ver vuelos de cuenca a quito, "
            "hay vuelos para mañana, vuelos directos, "
            "vuelos económicos para este viernes"
        )
    },
    {
        "name": "consultar_precio_vuelo",
        "category": "flight",
        "description": "Consulta el precio aproximado de un vuelo",
        "examples": (
            "cuánto cuesta un vuelo a guayaquil, "
            "precio del pasaje de quito a cuenca, "
            "cuánto vale volar a galápagos, "
            "precio aproximado del vuelo"
        )
    },
    {
        "name": "consultar_disponibilidad_vuelo",
        "category": "flight",
        "description": "Consulta la disponibilidad de asientos en un vuelo",
        "examples": (
            "hay asientos disponibles para hoy, "
            "todavía hay cupos en el vuelo, "
            "queda espacio en el avión, "
            "hay disponibilidad para mañana"
        )
    },
    {
        "name": "consultar_horarios_vuelo",
        "category": "flight",
        "description": "Consulta horarios de salida y llegada de vuelos",
        "examples": (
            "a qué hora sale el vuelo, "
            "horarios de vuelos de quito a guayaquil, "
            "a qué hora llega el avión, "
            "horarios disponibles para volar"
        )
    },
    {
        "name": "consultar_duracion_vuelo",
        "category": "flight",
        "description": "Consulta la duración estimada de un vuelo",
        "examples": (
            "cuánto dura el vuelo de quito a guayaquil, "
            "tiempo de vuelo a cuenca, "
            "cuántas horas dura el viaje en avión"
        )
    },
    {
        "name": "consultar_equipaje_vuelo",
        "category": "flight",
        "description": "Consulta información sobre equipaje permitido en vuelos",
        "examples": (
            "cuántas maletas puedo llevar, "
            "el vuelo incluye equipaje, "
            "puedo llevar maleta en bodega, "
            "equipaje permitido en el avión"
        )
    },
    {
        "name": "crear_reserva_vuelo",
        "category": "flight",
        "description": "Crea una reserva de vuelo",
        "examples": (
            "quiero reservar un vuelo, "
            "hacer una reserva de pasaje, "
            "resérvame un vuelo para mañana, "
            "quiero comprar el pasaje"
        )
    },
    {
        "name": "consultar_reserva_vuelo",
        "category": "flight",
        "description": "Consulta una reserva de vuelo existente",
        "examples": (
            "ver mi reserva de vuelo, "
            "consultar mi pasaje, "
            "estado de mi reserva, "
            "revisar mi vuelo reservado"
        )
    },
    {
        "name": "cancelar_reserva_vuelo",
        "category": "flight",
        "description": "Cancela una reserva de vuelo existente",
        "examples": (
            "cancelar mi vuelo, "
            "anular reserva de pasaje, "
            "ya no quiero viajar, "
            "cancelar el vuelo que compré"
        )
    },
    {
        "name": "buscar_ofertas_vuelos",
        "category": "flight",
        "description": "Busca ofertas y promociones disponibles en vuelos",
        "examples": (
            "ofertas de vuelos, "
            "promociones en pasajes, "
            "vuelos baratos, "
            "descuentos para volar"
        )
    }
]


In [4]:
def build_function_text(f):
    return f"{f['name']}: {f['description']}. Ejemplos: {f['examples']}"


In [11]:
va = {
        "name": "saludo",
        "category": "general",
        "description": "Saluda al usuario de forma amable",
        "examples": 
            "hola, buenos días, buenas tardes, buenas noches, "+
            "hola qué tal, hey, holaa, buenas, que mas?, que mas ve, como vas"
        
    }

In [12]:
texto = build_function_text(va)
texto

'saludo: Saluda al usuario de forma amable. Ejemplos: hola, buenos días, buenas tardes, buenas noches, hola qué tal, hey, holaa, buenas, que mas?, que mas ve, como vas'

In [13]:
embe = embedding_model.embed_query(texto)
embe

[0.011156040243804455,
 0.09064726531505585,
 0.005645496305078268,
 -0.04789124056696892,
 -0.06698110699653625,
 0.02720584347844124,
 0.060887470841407776,
 -0.001399331376887858,
 -0.005081229843199253,
 0.023580091074109077,
 0.06909625977277756,
 0.047676991671323776,
 -0.02008938230574131,
 -0.01762004941701889,
 0.030747240409255028,
 0.02692846953868866,
 -0.02144097164273262,
 0.002298863837495446,
 0.020916594192385674,
 0.05114097148180008,
 0.18066686391830444,
 0.0582885816693306,
 -0.0736328512430191,
 0.06705620139837265,
 -0.0348670594394207,
 0.07562526315450668,
 0.006515739485621452,
 0.029604606330394745,
 -0.012386877089738846,
 -0.11644450575113297,
 -0.02968531847000122,
 -0.0007903507794253528,
 0.14752686023712158,
 -0.03952546790242195,
 -0.012764996849000454,
 -0.030002491548657417,
 0.06593073159456253,
 -0.039789095520973206,
 -0.053127579391002655,
 0.05348673835396767,
 -0.07915028184652328,
 -0.02768106199800968,
 -0.02723303809762001,
 0.03186949715018

In [14]:
cur.execute("""
            INSERT INTO business_functions (name, description, examples, embedding, category)
        VALUES (%s, %s, %s, %s, %s)
            """, (va["name"], va["description"], va["examples"], embe, va["category"]))
conn.commit()

In [6]:
for f in functions:
    text = build_function_text(f)
    emb = embedding_model.embed_query(text)

    cur.execute(
        """
        INSERT INTO business_functions (name, description, examples, embedding, category)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (f["name"], f["description"], f["examples"], emb, f["category"])
    )

conn.commit()


In [ ]:
def select_function(query, top_k=3):
    query_emb = model.encode(query).tolist()
    cur.execute(
        """
        SELECT function_id, name,
               1 - (embedding <=> %s::vector) AS score
               
        FROM business_functions
        ORDER BY (embedding <=> %s::vector)
        LIMIT %s;
        """,
        (query_emb, query_emb, top_k)
    )

    results = cur.fetchall()

    # Guardar query
    best = results[0]
    cur.execute(
        """
        INSERT INTO query_log (query_text, query_embedding, selected_function_id, score)
        VALUES (%s, %s, %s, %s)
        """,
        (query, query_emb, best[0], best[2])
    )
    conn.commit()

    return results


In [24]:
def select_function_langchain(query, top_k=3):
    query_emb = embedding_model.embed_query(query)

    cur.execute(
        """
        SELECT function_id, name,
               1 - (embedding <=> %s::vector) AS score
        FROM business_functions
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
        """,
        (query_emb, query_emb, top_k)
    )
    return cur.fetchall()


In [19]:
results = select_function("quiero un plan de 3 días en quito")
for r in results:
    print(r)


(2, 'recomendar_itinerario', 0.725659505983622)
(10, 'recomendar_restaurantes', 0.486221700906754)
(1, 'buscar_destino', 0.469167560338974)


In [12]:
results = select_function("busco un hotel barato en el centro")
for r in results:
    print(r)


(4, 'buscar_hoteles', 0.737511060759755)
(1, 'buscar_destino', 0.546069161476766)
(3, 'buscar_vuelos', 0.542673305948773)


In [14]:
results = select_function("necesito vuelos económicos a guayaquil")
for r in results:
    print(r)

(3, 'buscar_vuelos', 0.571226256228276)
(8, 'consultar_reserva', 0.397763419217963)
(1, 'buscar_destino', 0.396816682720682)


In [15]:
results = select_function("recomiéndame restaurantes en quito")
for r in results:
    print(r)

(10, 'recomendar_restaurantes', 0.856026896475144)
(4, 'buscar_hoteles', 0.619809352170865)
(1, 'buscar_destino', 0.524280695660074)


In [27]:
results = select_function_langchain("recomiéndame restaurantes en quito")
for r in results:
    print(r)

(10, 'recomendar_restaurantes', 0.856026896475144)
(4, 'buscar_hoteles', 0.619809352170865)
(1, 'buscar_destino', 0.524280695660074)


In [1]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn

app = FastAPI()

@app.get("/")
def root():
    return {"msg": "API levantada desde un cuaderno 😎"}


In [4]:
nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop